# Vehicle Detection

In this project I'll write a software pipeline to detect vehicles in a video.

## Goals of Project

* Perform a Histogram of Oriented Gradients (HOG) feature extraction on a labeled training set of images and train a classifier Linear SVM classifier
* Optionally, you can also apply a color transform and append binned color features, as well as histograms of color, to your HOG feature vector. 
* Note: for those first two steps don't forget to normalize your features and randomize a selection for training and testing.
* Implement a sliding-window technique and use your trained classifier to search for vehicles in images.
* Run your pipeline on a video stream (start with the test_video.mp4 and later implement on full project_video.mp4) and create a heat map of recurring detections frame by frame to reject outliers and follow detected vehicles.
* Estimate a bounding box for vehicles detected.

# Data Exploration

There are two videos given first one `test_video.mp4` which can be used to test pipeline. Second one `project_video.mp4` is the video on which will be evaluated and pipeline should be able to detect vehicles reliably on that video.

Training images are split into **vehicle** and **non-vehicle** classes.

<figure>
 <img src="./illustrations/vehicle_01.png" width="64" alt="Vehicle 01" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Example of Vehicle</p> 
 </figcaption>
</figure>

<figure>
 <img src="./illustrations/vehicle_02.png" width="64" alt="Vehicle 02" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Example of Another Vehicle</p> 
 </figcaption>
</figure>

<figure>
 <img src="./illustrations/non_vehicle_01.png" width="64" alt="Non Vehicle 01" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Example of Non Vehicle</p> 
 </figcaption>
</figure>

<figure>
 <img src="./illustrations/non_vehicle_02.png" width="64" alt="Non Vehicle 02" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Example of Another Non Vehicle</p> 
 </figcaption>
</figure>




### Load Data

I have created a helpful script which downloads data from internet and unpacks it into `./data` directory

Next we need a way to load training images in python. My plan is to create 2 functions for loading training data. First one loads small subset of data and the second one loads the full set. Both of those functions should return data in same format and in this case let's define that the format should be **uint8 BGR** which is easy to use with OpenCV2.

in `dataset.py` i have created 2 functions `load_small()` and `load_full()` which are loading the small and full datasets respectively.
Those functions return features and labels and in this case feature is array of (N, 64, 64, 3) where **N** denotes the number of images. labels are simply **number 0 for non-vehicles** and **number 1 for vehicles**.



### Statistics

Let's briefly check quantity, shape and type of data.

In [2]:
import dataset

dataset.print_statistics()

Small data set:
	Number of vehicle images = 1196
	Number of non-vehicle images = 1125
	Shape of the image is  (64, 64, 3)




Full data set:
	Number of vehicle images = 8792
	Number of non-vehicle images = 8968
	Shape of the image is  (64, 64, 3)


# Feature Selection



In [3]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
import time
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import train_test_split
from dataset import *

# Define a function to compute binned color features  
def bin_spatial(img, size=(32, 32)):
    # Use cv2.resize().ravel() to create the feature vector
    features = cv2.resize(img, size).ravel() 
    # Return the feature vector
    return features

X, y = load_small()

features = []
for x in X:
    feat = bin_spatial((32,32))
    features.append(feat)

# Fit a per-column scaler
X_scaler = StandardScaler().fit(features)
# Apply the scaler to X
scaled_X = X_scaler.transform(features)

/home/lauri/miniconda3/envs/carnd-term1/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
